In [ ]:
# Transformers installation
!pip install transformers datasets --quiet
!pip install pytorchvideo --quiet
!pip install torch==1.11.0 torchvision==0.12.0 torchaudio==0.11.0 --quiet
!pip install evaluate --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 66.4 MB/s eta 0:00:00


In [ ]:
import argparse
import json
import urllib.request
from moviepy.video.io.VideoFileClip import VideoFileClip
import os
import pandas as pd
from huggingface_hub import notebook_login, login
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

notebook_login() # You need write access in huggingface

In [ ]:
from transformers import pipeline

video_cls = pipeline(model="markytools/videomae-base-finetuned-ucf101-subset", device="cuda")

In [ ]:
def fetch_loom_download_url(id):
    request = urllib.request.Request(
        url=f"https://www.loom.com/api/campaigns/sessions/{id}/transcoded-url",
        headers={},
        method="POST",
    )
    response = urllib.request.urlopen(request)
    body = response.read()
    content = json.loads(body.decode("utf-8"))
    url = content["url"]
    return url


def download_loom_video(url, filename):
    urllib.request.urlretrieve(url, filename)


def trim_video(input_file, output_file, start_time=1):
    with VideoFileClip(input_file) as video:
        trimmed_video = video.subclip(0, start_time)  # Trim the first 1 second
        trimmed_video.write_videofile(output_file, codec="mpeg4")


def parse_arguments():
    parser = argparse.ArgumentParser(
        prog="loom-dl", description="script to download loom.com videos"
    )
    parser.add_argument(
        "url", help="Url of the video in the format https://www.loom.com/share/[ID]"
    )
    parser.add_argument("-o", "--out", help="Path to output the file to")
    arguments = parser.parse_args()
    return arguments


def extract_id(url):
    return url.split("/")[-1]

def delete_file_if_exists(file_path):
    if os.path.isfile(file_path):
        os.remove(file_path)

def download_loom_video_to_file(url_str, trimmed_filename):
    # url_str = "https://www.loom.com/share/fb8f4b80277249a681087be933bf546c"
    id = extract_id(url_str)

    url = fetch_loom_download_url(id)
    filename = f"{id}.mp4"
    print(f"Loading video {id}")
    download_loom_video(url, filename)
    trim_video(filename, trimmed_filename)

    # Example usage
    delete_file_if_exists(filename)

# Function to color the cell red
def color_cell_red(sheet, row, col):
    red_fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")
    sheet.cell(row=row, column=col).fill = red_fill

# Function to remove the query part from the URL
def remove_query_from_url(url):
    return url.split('?')[0]

In [ ]:
# Read the Excel file
excel_file_path = '/content/linksTestOnly.xlsx'
df = pd.read_excel(excel_file_path)
df['video'] = df['video'].apply(remove_query_from_url)

# Iterate over each row in the dataframe
for index, row in df.iterrows():
    url = row['video']

    filename = "temp_vid.avi"
    # Call the download function
    download_loom_video_to_file(url_str=url, trimmed_filename=filename)
    output_pred = video_cls(filename)

    # Find the dictionary with the highest score
    highest_score_pred = max(output_pred, key=lambda x: x['score'])
    label = highest_score_pred['label']

    # Load the workbook and active sheet
    workbook = load_workbook(excel_file_path)
    sheet = workbook.active

    # If the label is "bad", color the cell red
    if label == "bad":
        color_cell_red(sheet, index + 2, 1)  # +2 because Excel rows start at 1, header row considered

    # Save the workbook after processing each video
    workbook.save(excel_file_path)